# Initialization

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm as tqdm

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import netCDF4

In [ ]:
import os.path
import glob

In [ ]:
# if incompact3d is configured with doubleprec then use np.float64
myfloat = np.float32

In [ ]:
cases = [2, 4, 5, 6, 7]

In [ ]:
description = {0: "Streamwise", 1: "Vertical", 2: "Spanwise"}

## Tools

In [ ]:
def read_prm_to_dict(path):
    '''
    This function reads the file BC-Plumes.prm from i3d and converts it into a Python dictionary
    
    Args:
        path: Path to folder that contains the file BC-Plumes.prm
    
    Returns:
        A dictionary containing flow parameters
    '''
    prm = {}
    prm["filename"] = path + '/BC-Plumes.prm'
    prm["datapath"] = path + '/data/'  # <- default value
    prm["x0ramp"] = 0.0  # <- default value
    f = open(prm["filename"], 'r')
    for line in f:
        line = line.partition('#')[0]  #Remove comments
        line = " ".join(line.split())  #Remove white spaces
        if line == '':  #Cycle if line is empty
            continue

        param = line.partition(' ')[0]
        value = line.partition(param + ' ')[-1]
        if value[0] == "'" and value[-1] == "'":  #String
            value = value[1:-1]
        elif " " in value:  #Array
            if "." in value:
                value = [float(i) for i in value.split(" ")] #Float
            else:
                value = [int(i) for i in value.split(" ")] #int
        else:  #Not Array
            if "." in value:
                value = float(value) #Float
            else:
                value = int(value) #int
        prm[param] = value
    f.close()
    return prm

In [ ]:
def x3d_readfield_2d(filename, n1, n2, n3=0, dtype=myfloat):
    '''
    This functions reads a binary field and returns it in a 2D numpy array, if
    n3 is not zero, it computes a spanwise-average and returns a 2D numpy array.
    
    Args:
        filename: File name.
        n1: Number of points in axis 0.
        n2: Number of points in axis 1.
        n3: Number of points in axis 2.
        dtype: Data type for Numpy array.
    
    Returns:
        A Numpy array with shape (n1, n2)
    
    Example:
        x3d_readfield_2d('./data/ux0010', 256, 64)
        x3d_readfield_2d('./data/phi10010', 256, 64, 16)
    '''
    if n3 == 0:  #2D
        return np.fromfile(filename, dtype=dtype).reshape((n1, n2),
                                                                 order='F')
    else:  #3D
        return np.average(np.fromfile(filename, dtype=dtype).reshape(
            (n1, n2, n3), order='F'),
                          axis=-2)

In [ ]:
def x3d_readfield_all_2d(target, n1, n2, n3=0, dtype=myfloat, engine='numpy'):
    '''
    This functions reads many binary fields and returns them in a stacked 3D array, if
    n3 is not zero, it computes a spanwise average and returns a 3D dask array.
    
    Args:
        Target: Pathname pattern.
        n1: Number of points in axis 0.
        n2: Number of points in axis 1.
        n3: Number of points in axis 2.
        dtype: Data type.
        engine: 'numpy' or 'dask'
    
    Returns:
        A array of shape (n1, n2, nfiles)
    
    Example:
        x3d_readfield_all_2d('./data/ux*', 256, 64)
        x3d_readfield_all_2d('./data/uy????', 256, 64, 16)
    '''
    #
    filenames = sorted(glob.glob(target))
    #
    if engine.lower() == 'dask':

        readfield = dask.delayed(x3d_readfield_2d, pure=True)

        lazy_fields = [
            readfield(file, n1, n2, n3, dtype=dtype) for file in filenames
        ]

        fields = [
            da.from_delayed(lazy_field, dtype=myfloat, shape=(n1, n2))
            for lazy_field in lazy_fields
        ]

        return da.stack(fields, axis=2)
    elif engine.lower() == 'numpy':
        fields = [
            x3d_readfield_2d(file, n1, n2, n3, dtype=dtype)
            for file in tqdm(filenames,
                             desc=f'Reading {os.path.split(target)[-1]}')
        ]

        return np.dstack(fields)
    else:
        print('invalid value for engine')
        return None

In [ ]:
def x3d_readfield_3d(filename, n1, n2, n3, dtype=myfloat):
    '''
    This functions reads a binary field and returns it in a 3D numpy array.
    
    Args:
        filename: File name.
        n1: Number of points in axis 0.
        n2: Number of points in axis 1.
        n3: Number of points in axis 2.
        dtype: Data type for Numpy array.
    
    Returns:
        A Numpy array with shape (n1, n2, n3)
    
    Example:
        x3d_readfield_2d('./data/ux0010', 256, 64, 16)
        x3d_readfield_2d('./data/phi10010', 256, 64, 16)
    '''
    return np.fromfile(filename, dtype=dtype).reshape((n1, n2, n3),
                                                                 order='F')

In [ ]:
def init_dataset(prm):
    '''
    This function initializes a xarray.dataset with the proper size and
    dimensions, in addition to some parameters and attributes.
    
    Args:
        prm: A dictionary containing flow parameters
    
    Returns:
        A xarray.Dataset
    '''
    ds = xr.Dataset(
        coords={
            'x':
                np.linspace(
                    0., prm["xlx"], num=prm["nx"], endpoint=True, dtype=myfloat)
                - prm["x0ramp"],
            'y':
                np.linspace(prm["yly"],
                            0.,
                            num=prm["ny"],
                            endpoint=True,
                            dtype=myfloat),
            'z':
                np.linspace(0.,
                            prm["zlz"],
                            num=prm["nz"],
                            endpoint=False,
                            dtype=myfloat),
            't':
                np.arange(
                    0, prm["ilast"] + 1, step=prm["iprocessing"], dtype=myfloat)
                * prm["dt"],
            'n':
                range(prm["nphi"])
        },
        attrs={
            'title':
                'The Plunging of Hyperpycnal Plumes on Tilted Bed by Three-Dimensional Large-Eddy Simulations',
            'authors': 'F. N. Schuch, J. H. Silvestrini, E. Meiburg & S. Laizet',
            'url': 'https://github.com/fschuch/the-plunging-flow-by-3D-LES',
            'doi': '10.5281/zenodo.3968993'
        })

    ds.x.attrs = {'name': 'Streamwise coordinate', 'long_name': r'$x_1$'}
    ds.y.attrs = {'name': 'Vertical coordinate', 'long_name': r'$x_2$'}
    ds.z.attrs = {'name': 'Spanwise coordinate', 'long_name': r'$x_3$'}
    ds.t.attrs = {'name': 'Time', 'long_name': r'$t$'}
    ds.n.attrs = {'name': 'Scalar fraction', 'long_name': r'$\ell$'}

    ds['uset'] = xr.DataArray(prm['uset'][:prm['nphi']],
                              dims=['n'],
                              coords=[ds.n],
                              attrs={
                                  'name': 'Settling Velocity',
                                  'long_name': r'$u_s$'
                              })

    ds['Ri0'] = xr.DataArray(prm['ri'][:prm['nphi']],
                             dims=['n'],
                             coords=[ds.n],
                             attrs={
                                 'name': 'Initial Richardson Number',
                                 'long_name': r'$Ri_0$'
                             })

    ds['Fr0'] = xr.DataArray(ds.Ri0**(-0.5),
                             dims=['n'],
                             coords=[ds.n],
                             attrs={
                                 'name': 'Initial densimetric Froude Number',
                                 'long_name': r'$Fr_0$'
                             })

    ds['Sc'] = xr.DataArray(prm['nsc'][:prm['nphi']],
                            dims=['n'],
                            coords=[ds.n],
                            attrs={
                                'name': 'Schmidt Number',
                                'long_name': r'$Sc$'
                            })

    ds['Re'] = xr.DataArray(prm['re'],
                            attrs={
                                'name': 'Reynolds Number',
                                'long_name': r'$Re$'
                            })
    '''
    Defining the position of the tilded bed, some operations are necessary in order
    to be consistent with the new coordinate system
    '''

    prm['xlx_pi'] -= prm['x0ramp']
    prm['xlx_pf'] -= prm['x0ramp']

    if 'declramp' in prm.keys():
        ds['S'] = xr.DataArray(prm['declramp'],
                               attrs={
                                   'name': 'Bed slope',
                                   'long_name': r'$S$'
                               })

        y0ramp = 1.
        layer = prm['yly'] - prm['layer']

        ramp = 1. + prm['declramp'] * ds.x

        ramp[ramp < y0ramp] = y0ramp
        ramp[ramp > layer] = layer

        ds['ramp'] = ramp

        ds.ramp.attrs['name'] = 'Bed position'
        ds.ramp.attrs['long_name'] = '$x_{2r}$'

        del ramp
    
    return ds

## Layer-averaged quantities

The complete spatio-temporal analysis of the relevant quantities is possible in a layer averaged context per width unit, that is computed according to the equations

$$
Uh(x_1,t) = \dfrac{1}{L_3} \int_0^{L_3} \int_{x_{2r}}^{x_{2i}} u_1(x_1,x_2,x_3,t) ~ dx_2 dx_3
$$
$$
U^2h(x_1,t) = \dfrac{1}{L_3} \int_0^{L_3} \int_{x_{2r}}^{x_{2i}} \big( u_1(x_1,x_2,x_3,t) \big)^2 ~ dx_2 dx_3
$$
$$
UCh(x_1,t) = \dfrac{1}{L_3} \int_0^{L_3} \int_{x_{2r}}^{x_{2i}} u_1 (x_1,x_2,x_3,t) ~ c_t (x_1,x_2,x_3,t) ~ dx_2 dx_3
$$

For the vertical integration, $x_{2r}$ represents the bed position and $x_{2i}$ represents the interface between the underflow turbidity current and the ambient fluid, considered in this work as the position where $u_1 c_t = 0.005$.
Then, they can be used to compute layer-averaged velocity $U = U^2h/Uh$, flow depth $H = (Uh)^2/U^2h)$, flow discharge $Q = Uh$, concentration $C = UCh/Uh$ and local densimetric Froude number $Fr$ (see [01-Computing-and-Plotting.ipynb](01-Computing-and-Plotting.ipynb)).

In [ ]:
def layer_averaged(ds, datapath):
    '''
    This function reads the raw binary fields from the disc and computes the
    Layer-averaged (LA) quantities, as described above.
    
    Args:
        ds: A base xarray.Dataset where LA variables are going to be stored.
        datapath: Path to the folder containg the simulated data.
    
    Returns:
        A xarray.Dataset with LA quantities
    '''
    # Tmp Initialization
    fields = xr.Dataset()

    fields['u'] = xr.DataArray(x3d_readfield_all_2d(
        os.path.join(datapath, 'xy-plane', 'ux????'), ds.x.size, ds.y.size),
                               dims=['x', 'y', 't'],
                               coords=[ds.x, ds.y,
                                       ds.t])
    fields['c'] = xr.concat([
        xr.DataArray(x3d_readfield_all_2d(
            os.path.join(datapath, 'xy-plane', f'phi{n+1}????'), ds.x.size,
            ds.y.size),
                     dims=['x', 'y', 't'],
                     coords=[ds.x, ds.y, ds.t]) for n in ds.n.to_index()
    ], 'n')
    #
    fields['uc'] = fields['u'] * fields['c'].sum('n')
    #
    mask = fields.uc >= 0.005
    #
    ds['Uh'] = (fields.u).where(mask, 0.0).integrate('y')
    ds['U2h'] = (fields.u**2.0).where(mask, 0.0).integrate('y')
    ds['UCh'] = (fields.u * fields.c).where(mask, 0.0).integrate('y')
    #
    ds.Uh.attrs = {'name': 'Layer-averaged Uh', 'long_name': r'$Uh$'}
    ds.U2h.attrs = {'name': 'Layer-averaged U2h', 'long_name': r'$U^2h$'}
    ds.UCh.attrs = {'name': 'Layer-averaged UCh', 'long_name': r'$UCh$'}
    #
    return ds

# Code

## LA quantities

The block bellow computes and writes to the disc a data set containing the layer-averaged quantities, in addition to spanwise-averaged bed shear velocity and spanwise-averaged deposition rate.

In [ ]:
for sim in tqdm(cases, desc='Case'):
    path = "D:/Simulacoes_Arquivadas/run3/incompact3d_plumes_edition/exp-" + str(sim)
    #
    prm = read_prm_to_dict(path)
    #
    ds = init_dataset(prm)
    #
    ds = layer_averaged(ds, prm['datapath'])
    #
    ds['utau'] = xr.DataArray(
        x3d_readfield_all_2d(os.path.join(prm['datapath'], 'xz-plane', 'utau*'),
                             ds.x.size, ds.z.size),
        dims=['x', 'z', 't'],
        coords=[ds.x, ds.z, ds.t],
    ).mean('z')
    ds.utau.attrs = {'name': 'Bed shear velocity', 'long_name': r'$u_\tau$'}
    #
    ds['dep'] = xr.concat([
        xr.DataArray(x3d_readfield_all_2d(
            os.path.join(prm['datapath'], 'xz-plane', f'dep{n+1}*'), ds.x.size,
            ds.z.size),
                     dims=['x', 'z', 't'],
                     coords=[ds.x, ds.z, ds.t]).mean('z')
        for n in ds.n.to_index()
    ], 'n')
    ds.dep.attrs = {'name': 'Deposition rate', 'long_name': r'$\dot{D}$'}
    # Cut to test section and save file
    ds.sel(x=slice(prm['xlx_pi'], prm['xlx_pf'])).to_netcdf(f'LA-case-{sim}.nc')

## xy-planes

This block converts the raw binary containing the spanwise-averaged planes to NefCDF.

In [ ]:
for sim in tqdm(cases, desc='Case'):
    path = os.path.normcase(
        "D:/Simulacoes_Arquivadas/run3/incompact3d_plumes_edition/exp-"
        + str(sim))
    #
    prm = read_prm_to_dict(path)
    #
    ds = init_dataset(prm)
    #
    for i, var in enumerate('ux uy uz'.split()):

        ds[var] = xr.DataArray(x3d_readfield_all_2d(
        os.path.join(path, 'data', 'xy-plane', f'{var}????'), ds.x.size, ds.y.size),
                               dims=['x', 'y', 't'],
                               coords=[ds.x, ds.y,ds.t],
                               attrs={
                                   'name': f"{description.get(i,'')} Velocity",
                                   'long_name': fr"$u_{i+1}$"
                               })
    #
    ds['phi'] = xr.concat([
        xr.DataArray(x3d_readfield_all_2d(
            os.path.join(path, 'data', 'xy-plane', f'phi{n+1}????'), ds.x.size,
            ds.y.size),
                     dims=['x', 'y', 't'],
                     coords=[ds.x, ds.y, ds.t]) for n in ds.n.to_index()
    ], 'n')
    #
    ds['phi'].attrs['name'] = 'Concentration Field'
    ds['phi'].attrs['long_name'] = r'$\varphi$'
    # Cut to test section and save file
    # Cut in time because of the size limitation at Zenodo
    ds.sel(x=slice(prm['xlx_pi'], prm['xlx_pf']),t=slice(None,None,2)).to_netcdf(f'xy-planes-case-{sim}.nc')

## xz-planes

This block converts the raw binary containing the depth-averaged planes to NefCDF.

In [ ]:
for sim in tqdm(cases, desc='Case'):
    path = os.path.normcase(
        "D:/Simulacoes_Arquivadas/run3/incompact3d_plumes_edition/exp-"
        + str(sim))
    #
    prm = read_prm_to_dict(path)
    #
    ds = init_dataset(prm)
    #
    for i, var in enumerate('ux uy uz'.split()):
        ds[var] = xr.DataArray(x3d_readfield_all_2d(
        os.path.join(path, 'data', 'xz-plane', f'{var}????'), ds.x.size, ds.z.size),
                               dims=['x', 'z', 't'],
                               coords=[ds.x, ds.z,ds.t],
                               attrs={
                                   'name': f"{description.get(i,'')} Velocity",
                                   'long_name': fr"$u_{i+1}$"
                               })
    #
    ds['phi'] = xr.concat([
        xr.DataArray(x3d_readfield_all_2d(
            os.path.join(path, 'data', 'xz-plane', f'phi{n+1}????'), ds.x.size,
            ds.z.size),
                     dims=['x', 'z', 't'],
                     coords=[ds.x, ds.z, ds.t]) for n in ds.n.to_index()
    ], 'n')
    #
    ds['phi'].attrs['name'] = 'Concentration Field'
    ds['phi'].attrs['long_name'] = r'$\varphi$'
    # Cut to test section and save file
    # Cut in time because of the size limitation at Zenodo
    ds.sel(x=slice(prm['xlx_pi'], prm['xlx_pf']),t=slice(None,None,2)).to_netcdf(f'xz-planes-case-{sim}.nc')

## 3D-Snapshots

This block converts the raw binary containing the 3D fields to NefCDF.

In [ ]:
times = np.array([250,
                  500,
                  750,
                  1000,
                  1250,
                  1500,
                  1750,
                  2000,
                  3000,
                  4000,
                  5000,
                  6000],
                 dtype=myfloat)

for sim in tqdm(cases, desc='Case'):
    path = os.path.normcase(
        "D:/Simulacoes_Arquivadas/run3/incompact3d_plumes_edition/exp-"
        + str(sim))
    #
    prm = read_prm_to_dict(path)
    #
    ds = init_dataset(prm)
    #
    ds['t'] = times
    ds.t.attrs = {'name': 'Time', 'long_name': r'$t$'}
    #
    itimes = np.array(times / prm['dt'] / prm['imodulo'], dtype=np.int32)
    #
    for i, var in enumerate('ux uy uz'.split()):
        ds[var] = xr.DataArray(myfloat(0.),
                               coords=[ds.x, ds.y, ds.z, ds.t],
                               attrs={
                                   'name': f"{description.get(i,'')} Velocity",
                                   'long_name': fr"$u_{i+1}$"
                               })
        for k, t in enumerate(ds.t):
            ds[var][dict(t=k)] += x3d_readfield_3d(
                os.path.join(path, 'data', '3d-snapshot',
                             var + str(itimes[k]).zfill(4)), ds.x.size,
                ds.y.size, ds.z.size)
    #
    ds['phi'] = xr.DataArray(myfloat(0.),
                               coords=[ds.x, ds.y, ds.z, ds.t, ds.n],
                               attrs={
                               'name': 'Concentration Field',
                               'long_name': r'$\varphi$'
                           })
    #
    var = 'phi'
    for k, t in enumerate(ds.t):
        for n in ds.n.values:
            ds[var][dict(t=k,n=n)] += x3d_readfield_3d(
                os.path.join(path, 'data', '3d-snapshot',
                             var + str(n+1) + str(itimes[k]).zfill(4)), ds.x.size,
                ds.y.size, ds.z.size)
    # Cut to test section, clear bellow channel's bed and save file
    ds.where(ds.y < ds.ramp, 0.0).sel(x=slice(prm['xlx_pi'], prm['xlx_pf'])).to_netcdf(f'3d-case-{sim}.nc')